In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

plt.style.use('dark_background')

In [2]:
# Masses of the planets
m_1 = 10
m_2 = 20
m_3 = 30

# Starting coordinates and velocities for planets
p1_start = np.array([-10, 10, -11])
v1_start = np.array([-3, 0, 0])

p2_start = np.array([0, 0, 0])
v2_start = np.array([0, 0, 0])

p3_start = np.array([10, 10, 12])
v3_start = np.array([3, 0, 0])


In [3]:
def accelerations(p1, p2, p3):
    """
    A function to calculate the accelerations of each planet 
    based on the positions of the three bodies.
    """
    planet_1_dv = -9.8 * m_2 * (p1 - p2)/(np.linalg.norm(p1 - p2)**3) - \
                  9.8 * m_3 * (p1 - p3)/(np.linalg.norm(p1 - p3)**3)

    planet_2_dv = -9.8 * m_3 * (p2 - p3)/(np.linalg.norm(p2 - p3)**3) - \
                  9.8 * m_1 * (p2 - p1)/(np.linalg.norm(p2 - p1)**3)

    planet_3_dv = -9.8 * m_1 * (p3 - p1)/(np.linalg.norm(p3 - p1)**3) - \
                  9.8 * m_2 * (p3 - p2)/(np.linalg.norm(p3 - p2)**3)

    return planet_1_dv, planet_2_dv, planet_3_dv


In [4]:
# Number of different initial conditions
num_initial_conditions = 100

# Time step and number of steps for each simulation
delta_t = 0.001
steps = 2000  # Can adjust for performance


In [5]:
def generate_initial_conditions():
    p1_start = np.random.uniform(-20, 20, size=3)
    v1_start = np.random.uniform(-5, 5, size=3)
    p2_start = np.random.uniform(-20, 20, size=3)
    v2_start = np.random.uniform(-5, 5, size=3)
    p3_start = np.random.uniform(-20, 20, size=3)
    v3_start = np.random.uniform(-5, 5, size=3)
    return p1_start, v1_start, p2_start, v2_start, p3_start, v3_start


In [6]:
# Lists to store the dataset for all initial conditions
all_data_input = []
all_data_output = []

In [10]:
def plot_trajectory(p1, p2, p3, condition_number):
    """
    Plot the 3D trajectories of the three bodies for a given condition number and save the plot as an image.
    """
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot(p1[:, 0], p1[:, 1], p1[:, 2], label='Body 1')
    ax.plot(p2[:, 0], p2[:, 1], p2[:, 2], label='Body 2')
    ax.plot(p3[:, 0], p3[:, 1], p3[:, 2], label='Body 3')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()

    # Save the plot for this condition
    plt.savefig(f"trajectory_condition_{condition_number}.png")
    plt.close()  # Close the figure to avoid memory leaks


In [7]:
# Loop over each initial condition
for condition in range(num_initial_conditions):
    # Generate random initial positions and velocities
    p1_start, v1_start, p2_start, v2_start, p3_start, v3_start = generate_initial_conditions()

    # Initialize trajectory arrays for this particular set
    p1 = np.zeros((steps, 3))
    v1 = np.zeros((steps, 3))
    p2 = np.zeros((steps, 3))
    v2 = np.zeros((steps, 3))
    p3 = np.zeros((steps, 3))
    v3 = np.zeros((steps, 3))

    # Set initial positions and velocities
    p1[0], p2[0], p3[0] = p1_start, p2_start, p3_start
    v1[0], v2[0], v3[0] = v1_start, v2_start, v3_start

    # Evolve the system over time
    for i in range(steps - 1):
        # Calculate accelerations
        dv1, dv2, dv3 = accelerations(p1[i], p2[i], p3[i])

        # Update velocities
        v1[i + 1] = v1[i] + dv1 * delta_t
        v2[i + 1] = v2[i] + dv2 * delta_t
        v3[i + 1] = v3[i] + dv3 * delta_t

        # Update positions
        p1[i + 1] = p1[i] + v1[i] * delta_t
        p2[i + 1] = p2[i] + v2[i] * delta_t
        p3[i + 1] = p3[i] + v3[i] * delta_t

        # Store current positions and velocities as input
        current_input = np.concatenate((p1[i], v1[i], p2[i], v2[i], p3[i], v3[i]))
        all_data_input.append(current_input)

        # Store next step positions and velocities as output
        next_output = np.concatenate((p1[i + 1], v1[i + 1], p2[i + 1], v2[i + 1], p3[i + 1], v3[i + 1]))
        all_data_output.append(next_output)
        
    plot_trajectory(p1, p2, p3, condition)

In [8]:
# Convert lists to DataFrame
df_input = pd.DataFrame(all_data_input)
df_output = pd.DataFrame(all_data_output)

# Save the datasets to CSV files
df_input.to_csv('three_body_input.csv', index=False)
df_output.to_csv('three_body_output.csv', index=False)